In [10]:
%load_ext autoreload
%autoreload 2 
import sys
sys.path.append("../../../xapres/")
import xapres as xa
import numpy as np
import xarray as xr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
fd = xa.load.from_dats()
new = fd.load_all(directory='../../data/sample/single_dat_file/', legacy_fft=False, addProfileToDs_kwargs={'crop_chirp_start':0,'crop_chirp_end':1})
new

<xarray.Dataset> Size: 97MB
Dimensions:          (time: 1, chirp_time: 40001, chirp_num: 100,
                      attenuator_setting_pair: 1, profile_range: 39999)
Coordinates:
  * time             (time) datetime64[ns] 8B 2023-01-05T03:16:15
  * chirp_time       (chirp_time) float64 320kB 0.0 2.5e-05 5e-05 ... 1.0 1.0
  * chirp_num        (chirp_num) int64 800B 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
    filename         (time) <U57 228B '../../data/sample/single_dat_file/DATA...
    burst_number     (time) int64 8B 0
    AFGain           (attenuator_setting_pair) int64 8B -4
    attenuator       (attenuator_setting_pair) float64 8B 22.0
    orientation      (time) <U7 28B 'unknown'
  * profile_range    (profile_range) float64 320kB 0.0 0.2103 ... 8.411e+03
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 32MB ...
    latitude         (time) float64 8B 0.0
    longitude        (time) float64 8B 0.0
    battery_voltage  (time) float64 8B 0.0
    temperature_1    (time) float64 8B 2.891
    temperature_2    (time) float64 8B 511.8
    profile          (time, chirp_num, attenuator_setting_pair, profile_range) complex128 64MB ...
Attributes:
    constants:   {'c': 300000000.0, 'K': 200000000.0, 'f_1': 200000000, 'f_2'...
    processing:  Created on 2024-10-14 16:00:54.883945

temperature_2 is 511.8. This is clearly incorrect.



In [26]:
new['temperature_2'].values

array([511.828])

 In the matlab code this is corrected for:

This is the code from the matlab version ([LoadBurstRMB3.m](https://github.com/ldeo-glaciology/phase-sensitive-radar-processing/blob/b09870189b1655b2745d751e58c06b754320f6ab/code/Matlab_fmcwSuite2018/LoadBurstRMB3.m#L119C1-L119C46)): 
```
if vdat.Code == 0
% Clean temperature record (wrong data type?)
bti1 = find(vdat.Temperature_1>300);
if ~isempty(bti1)
vdat.Temperature_1(bti1) = vdat.Temperature_1(bti1)-512;
end
bti2 = find(vdat.Temperature_2>300);
vdat.Temperature_2(bti2) = vdat.Temperature_2(bti2)-512;
end
```

In [27]:
new_Tcorrected = new

T1 = new['temperature_1']
T2 = new['temperature_2']

new_Tcorrected['temperature_1'] = xr.where(T1>300, T1-512, T1)
new_Tcorrected['temperature_2'] = xr.where(T2>300, T2-512, T2)
new_Tcorrected['temperature_2'].values

array([-0.172])

In [29]:
def correct_temperature(T):
    return xr.where(T>300, T-512, T)

In [32]:
def correct_temperature(self, threshold=300, correction = -512):
    T1 = self['temperature_1']
    T2 = self['temperature_2']

    self['temperature_1'] = xr.where(T1>threshold, T1+correction, T1)
    self['temperature_2'] = xr.where(T2>threshold, T2+correction, T2)    


In [35]:
correct_temperature(new)

In [36]:
new

<xarray.Dataset> Size: 97MB
Dimensions:          (time: 1, chirp_time: 40001, chirp_num: 100,
                      attenuator_setting_pair: 1, profile_range: 39999)
Coordinates:
  * time             (time) datetime64[ns] 8B 2023-01-05T03:16:15
  * chirp_time       (chirp_time) float64 320kB 0.0 2.5e-05 5e-05 ... 1.0 1.0
  * chirp_num        (chirp_num) int64 800B 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
    filename         (time) <U57 228B '../../data/sample/single_dat_file/DATA...
    burst_number     (time) int64 8B 0
    AFGain           (attenuator_setting_pair) int64 8B -4
    attenuator       (attenuator_setting_pair) float64 8B 22.0
    orientation      (time) <U7 28B 'unknown'
  * profile_range    (profile_range) float64 320kB 0.0 0.2103 ... 8.411e+03
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 32MB ...
    latitude         (time) float64 8B 0.0
    longitude        (time) float64 8B 0.0
    battery_voltage  (time) float64 8B 0.0
    temperature_1    (time) float64 8B 2.891
    temperature_2    (time) float64 8B -0.172
    profile          (time, chirp_num, attenuator_setting_pair, profile_range) complex128 64MB ...
Attributes:
    constants:   {'c': 300000000.0, 'K': 200000000.0, 'f_1': 200000000, 'f_2'...
    processing:  Created on 2024-10-14 16:00:54.883945

In [39]:
ds = fd.load_all(directory='../../data/sample/single_dat_file/', legacy_fft=False, addProfileToDs_kwargs={'crop_chirp_start':0,'crop_chirp_end':1})
ds

<xarray.Dataset> Size: 97MB
Dimensions:          (time: 1, chirp_time: 40001, chirp_num: 100,
                      attenuator_setting_pair: 1, profile_range: 39999)
Coordinates:
  * time             (time) datetime64[ns] 8B 2023-01-05T03:16:15
  * chirp_time       (chirp_time) float64 320kB 0.0 2.5e-05 5e-05 ... 1.0 1.0
  * chirp_num        (chirp_num) int64 800B 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
    filename         (time) <U57 228B '../../data/sample/single_dat_file/DATA...
    burst_number     (time) int64 8B 0
    AFGain           (attenuator_setting_pair) int64 8B -4
    attenuator       (attenuator_setting_pair) float64 8B 22.0
    orientation      (time) <U7 28B 'unknown'
  * profile_range    (profile_range) float64 320kB 0.0 0.2103 ... 8.411e+03
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 32MB ...
    latitude         (time) float64 8B 0.0
    longitude        (time) float64 8B 0.0
    battery_voltage  (time) float64 8B 0.0
    temperature_1    (time) float64 8B 2.891
    temperature_2    (time) float64 8B -0.172
    profile          (time, chirp_num, attenuator_setting_pair, profile_range) complex128 64MB ...
Attributes:
    constants:   {'c': 300000000.0, 'K': 200000000.0, 'f_1': 200000000, 'f_2'...
    processing:  Created on 2024-10-14 16:08:11.031640